In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

### Importing Libraries

### Importing Datasets

In [ ]:
# Load your dataset
data = pd.read_excel('finale.xlsx')
data.head(3)

### Calculating and Creating the Activity Column

##### The logic has been updated to classify members as Active if they meet either of the following conditions:

1. *At least 8 gym+pool visits OR* *At least 3 total bookings.*

In [ ]:
# Logic to create the 'Activity' column
data['Activity'] = ((data['gym_visits'] + data['pool_visits'] >= 20) | (data['total_bookings'] >= 6)).map({True: 'Active', False: 'Inactive'})

# Displaying the first few rows to confirm the new column
data.head()

# Prediction Model Building

### CatBoost

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Example DataFrame
df_1 = data.copy()

# Define features and target
features = ['pool_visits', 'mem_length', 'mem_category', 'age', 'gender', 'class_bookings', 'alt_bookings',
            'oth_bookings', 'squash_bookings', 'price_level']
categorical_columns = ['mem_category', 'price_level', 'gender']
df_1[categorical_columns] = df_1[categorical_columns].astype('category')

X = df_1[features]
y = df_1['Activity'].map({'Active': 1, 'Inactive': 0})
y = y.fillna(0)  # Filling any remaining NaN values

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Specify categorical features by their column indices
cat_features = [X.columns.get_loc(col) for col in categorical_columns]

# Initialize the CatBoost model
model = CatBoostClassifier(
    iterations=100,            # Number of boosting iterations
    learning_rate=0.1,         # Learning rate
    depth=6,                   # Depth of the tree
    cat_features=cat_features, # Specify categorical features
    verbose=10                 # Verbosity
)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the training set
y_train_pred = model.predict(X_train)

# Evaluate the model on the training set
# train_accuracy = accuracy_score(y_train, y_train_pred)
# train_report = classification_report(y_train, y_train_pred)

# print("\nTraining Set Accuracy: {:.2f}".format(train_accuracy))
# print("\nTraining Set Classification Report:")
# print(train_report)

# Make predictions on the test set
y_test_pred = model.predict(X_test)

Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
test_report = classification_report(y_test, y_test_pred)

print("\nTest Set Accuracy: {:.2f}".format(test_accuracy))
print("\nTest Set Classification Report:")
print(test_report)


In [ ]:
import pickle

# Assuming 'model' is your trained CatBoost model
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)


### User input and prediction

In [ ]:
# def predict_activity(model, pool_visits, mem_length, mem_category, age, gender, 
#                      class_bookings, alt_bookings, oth_bookings, squash_bookings, price_level):
    
#     input_data = pd.DataFrame({
#         'pool_visits': [pool_visits],
#         'mem_length': [mem_length],
#         'mem_category': [mem_category],
#         'age': [age],
#         'gender': [gender],
#         'class_bookings': [class_bookings],
#         'alt_bookings': [alt_bookings],
#         'oth_bookings': [oth_bookings],
#         'squash_bookings': [squash_bookings],
#         'price_level': [price_level]
#     })
    
#     categorical_columns = ['mem_category', 'price_level', 'gender']
#     input_data[categorical_columns] = input_data[categorical_columns].astype('category')
    
#     prediction = model.predict(input_data)
    
#     activity_type = 'Active' if prediction[0] == 1 else 'Inactive'
    
#     return activity_type

# def print_prediction_output(predicted_activity, pool_visits, mem_length, mem_category, age, gender, 
#                             class_bookings, alt_bookings, oth_bookings, squash_bookings, price_level):
#     # Create a nicely formatted template for the output
#     print(" ")
#     print("=" * 50)
#     print(" " * 15 + "ACTIVITY PREDICTION")
#     print("=" * 50)
    
#     # Display input data in a tabular format
#     print(f"{'Pool Visits:':<20}{pool_visits}")
#     print(f"{'Membership Length:':<20}{mem_length} days")
#     print(f"{'Membership Category:':<20}{mem_category}")
#     print(f"{'Age:':<20}{age} years")
#     print(f"{'Gender:':<20}{gender}")
#     print(f"{'Class Bookings:':<20}{class_bookings}")
#     print(f"{'Alternative Bookings:':<20}{alt_bookings}")
#     print(f"{'Other Bookings:':<20}{oth_bookings}")
#     print(f"{'Squash Bookings:':<20}{squash_bookings}")
#     print(f"{'Price Level:':<20}{price_level}")
    
#     print("-" * 50)
    
#     # Display the predicted activity in a prominent way
#     print(f"{'Predicted Activity Type:': <20} {predicted_activity}")
    
#     print("=" * 50)
#     print(" " * 10 + "Prediction Completed")
#     print("=" * 50)

# # Asking for user inputs
# print("=" * 50)
# print(" "*15 + "Initiating Prediction Model")
# print("=" * 50)
# pool_visits = int(input("Enter the number of pool visits: "))
# mem_length = int(input("Enter the length of membership (in days): "))
# mem_category = input("Enter the membership category (e.g., peak, off-peak, others): ")
# age = int(input("Enter the user's age: "))
# gender = input("Enter the user's gender (Male/Female): ")
# class_bookings = int(input("Enter the number of class bookings: "))
# alt_bookings = int(input("Enter the number of alternative bookings: "))
# oth_bookings = int(input("Enter the number of other bookings: "))
# squash_bookings = int(input("Enter the number of squash bookings: "))
# price_level = input("Enter the price level (e.g., student, alumni, others, community, staff, junior): ")

# # Making the prediction
# predicted_activity = predict_activity(
#     model, 
#     pool_visits=pool_visits, 
#     mem_length=mem_length, 
#     mem_category=mem_category, 
#     age=age, 
#     gender=gender, 
#     class_bookings=class_bookings, 
#     alt_bookings=alt_bookings, 
#     oth_bookings=oth_bookings, 
#     squash_bookings=squash_bookings, 
#     price_level=price_level
# )

# # Printing the output with the design template
# print_prediction_output(
#     predicted_activity, 
#     pool_visits, 
#     mem_length, 
#     mem_category, 
#     age, 
#     gender, 
#     class_bookings, 
#     alt_bookings, 
#     oth_bookings, 
#     squash_bookings, 
#     price_level
# )


In [ ]:
# model.save_model('catboost_model.cbm')